<a href="https://colab.research.google.com/github/c0desh1n0b1/Data-Science/blob/Linear-Regression(Practical(Car-Price-Prediction))/Car_Prediction(Linear_Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
import io

In [3]:
from google.colab import files
uploaded = files.upload()



Saving quikr_car.csv to quikr_car.csv


In [ ]:
car = pd.read_csv(io.BytesIO(uploaded['quikr_car.csv']))
#Convert The csv to a pandas Dataset


In [9]:
car.head()
#Show the first 5 data in the set

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [7]:
car.shape
#6 Columns with 892 Rows 

(892, 6)

In [ ]:
car.info()

In [ ]:
car['year'].unique()

In [ ]:
car['Price'].unique()

In [ ]:
car['company'].unique()

In [ ]:
car['fuel_type'].unique()

In [ ]:
car['kms_driven'].unique()

In [ ]:
car['name'].unique()

#Quality of the dataset
1. The year has a lot of non values
2. Convert Year to integer
3. The price contains an ask for price value
4. The price neeeds to be converted to int
5. The fuel type has a non value
6. The km has a non value
7. Convert kms from object to integer
8. In the name store the first 3 words.


#Data Cleaning

In [10]:
backup = car.copy()
#Save a backup of the cardataset

In [11]:
car=car[car['year'].str.isnumeric()]
#Check whether the car is year is numeric

In [12]:
car['year']=car['year'].astype(int)
#Convert from Object to integer

In [13]:
car = car[car['Price'] != "Ask For Price"]
#Exclude Ask for Price

In [14]:
car['Price']=car['Price'].str.replace(',','').astype(int)
#Remove the comma then convert price to object

In [15]:
car = car[car['kms_driven'] != 'Petrol']
#Remove the Petrol Data section

In [16]:
car['kms_driven']=car['kms_driven'].str.split().str.get(0).str.replace(',','').astype(int)
#Split the string
#Remove the comma and kms
#Convert to int

In [17]:
car=car[~car['fuel_type'].isna()]
#Check what data type has no value

In [18]:
car['name'] =car['name'].str.split(' ').str.slice(0,3).str.join(' ')
#In the name, split the string, collect the first 3 string names and then join 

In [19]:
car.reset_index(drop=True)
#Reset the index of the car dataframe

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
811,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
812,Tata Indica V2,Tata,2009,110000,30000,Diesel
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
814,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [ ]:
car.describe()

In [21]:
car = car[car['Price']<6e6].reset_index(drop=True)

In [20]:
car.to_csv('Cleaned_set.csv')

# Model Building



In [22]:
X = car.drop(columns='Price')
y = car['Price']
#Drop the Price column which I assigned to y axis

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
#From the sklearn model, import the train test split where we will split the train test, 80% train 20 test%

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
#Sklearn score function to determine the regression score
from sklearn.preprocessing import OneHotEncoder
#Onehot encoding for categorical data
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
#Construct a pipeline from the given estimators
#Modules used for the model building

In [25]:
onehot = OneHotEncoder()
onehot.fit(X[['name','company','fuel_type']])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [26]:
column_trans = make_column_transformer((OneHotEncoder(categories=onehot.categories_),['name','company','fuel_type']),remainder='passthrough')
#Encoding the columns where the remainder is passthrough which means any columns that were not transformed will be automatically passed throuugh.

In [27]:
regress = LinearRegression()

In [28]:
pipe = make_pipeline(column_trans,regress)
#Makepipeline will connstruct a pipeline from given estimators, regression and the columns 

In [ ]:
pipe.fit(X_train,y_train)

In [30]:
y_pred = pipe.predict(X_test)
#prediction of the price based on X training dataset used.

In [38]:
r2_score(y_test,y_pred)
#Coefficient Score 

0.5747854691304858

In [81]:
score=[]
for i in range(1000):
  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
  regress=LinearRegression()
  pipe = make_pipeline(column_trans,regress)
  pipe.fit(X_train,y_train)
  y_pred=pipe.predict( X_test)
  score.append('%.3f'% r2_score(y_test,y_pred))
  #1000 iterations of the train test and predict process.
  #Append the different scores into a list 

In [ ]:
print(np.argmax(score))
score[np.argmax(score)]
#Returns the maximum indies of values, in this case the highest index is 661 where the r2score is 0.890

In [73]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(score))
regress=LinearRegression()
pipe = make_pipeline(column_trans,regress)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)
#Use the highest r2 score to fit and predict 

0.8897767222258609

In [83]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,0,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))
#This is a predict price of a data sample

array([407032.80803756])